In [1]:
import os 
import glob as glob 
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [2]:
dir_path = r"C:\Users\jinyo\Downloads\OneDrive_1_11-7-2024"

files = sorted(glob.glob(os.path.join(dir_path, "*.csv")))
files

['C:\\Users\\jinyo\\Downloads\\OneDrive_1_11-7-2024\\train_features.csv',
 'C:\\Users\\jinyo\\Downloads\\OneDrive_1_11-7-2024\\train_labels.csv']

In [3]:
X_df = pd.read_csv(files[0])
y_df = pd.read_csv(files[1])

In [4]:
print(len(X_df))
print(len(y_df))

3276
4343


In [5]:
X_df = X_df.sort_values(by="uid")
y_df = y_df.sort_values(by="uid")

In [6]:
ids = sorted(set(y_df['uid'].values))
print(len(ids))

for id in ids:
    print(np.where(y_df['uid']==id))

3276
(array([0], dtype=int64),)
(array([1], dtype=int64),)
(array([2, 3], dtype=int64),)
(array([4], dtype=int64),)
(array([5], dtype=int64),)
(array([6], dtype=int64),)
(array([7], dtype=int64),)
(array([8], dtype=int64),)
(array([9], dtype=int64),)
(array([10], dtype=int64),)
(array([11], dtype=int64),)
(array([12], dtype=int64),)
(array([13, 14], dtype=int64),)
(array([15, 16], dtype=int64),)
(array([17], dtype=int64),)
(array([18], dtype=int64),)
(array([19], dtype=int64),)
(array([20, 21], dtype=int64),)
(array([22], dtype=int64),)
(array([23], dtype=int64),)
(array([24], dtype=int64),)
(array([25], dtype=int64),)
(array([26], dtype=int64),)
(array([27], dtype=int64),)
(array([28], dtype=int64),)
(array([29], dtype=int64),)
(array([30], dtype=int64),)
(array([31], dtype=int64),)
(array([32], dtype=int64),)
(array([33, 34], dtype=int64),)
(array([35], dtype=int64),)
(array([36], dtype=int64),)
(array([37], dtype=int64),)
(array([38], dtype=int64),)
(array([39], dtype=int64),)
(arra

In [7]:
testing_df = X_df.drop(columns=["uid"])

In [8]:
missing_col = []
for column in testing_df.columns:
    if testing_df[column].isna().sum()/len(testing_df) >= 0.7:
        print(column)
        print(testing_df[column].isna().sum()/len(testing_df))
        print()
        missing_col.append(column)
print(missing_col)

rjlocc_m_03
0.9050671550671551

rjob_end_03
0.8965201465201466

rjobend_reason_03
0.8937728937728938

rjob_end_12
0.8556166056166056

rjobend_reason_12
0.851037851037851

a16a_12
0.9926739926739927

a21_12
0.9871794871794872

a22_12
0.989010989010989

a33b_12
0.9871794871794872

['rjlocc_m_03', 'rjob_end_03', 'rjobend_reason_03', 'rjob_end_12', 'rjobend_reason_12', 'a16a_12', 'a21_12', 'a22_12', 'a33b_12']


In [9]:
testing_df2 = testing_df.drop(columns=missing_col)

In [10]:
dtypes = []
for column in testing_df2.columns:
    dtypes.append(testing_df2[column].dtype)
print(set(dtypes))

{dtype('O'), dtype('float64')}


In [11]:
for column in testing_df2.columns:
    if testing_df2[column].dtype == "object":
        print(column)
        print(set(testing_df2[column].values))
        print()

age_03
{'3. 70–79', '0. 49 or younger', '4. 80+', nan, '2. 60–69', '1. 50–59'}

urban_03
{'0. <100,000', nan, '1. 100,000+'}

married_03
{'3. Widowed', '2. Separated or divorced', '1. Married or in civil union', '4. Single', nan}

edu_gru_03
{'0. No education', '2. 6 years', '3. 7–9 years', '4. 10+ years', nan, '1. 1–5 years'}

n_living_child_03
{'1. 1 or 2', '3. 5 or 6', nan, '0. No children', '2. 3 or 4', '4. 7+'}

glob_hlth_03
{'3. Good', '2. Very good', '5. Poor', nan, '4. Fair', '1. Excellent'}

bmi_03
{'4. Obese', '1. Underweight', '5. Morbidly obese', nan, '2. Normal weight', '3. Overweight'}

decis_famil_03
{'1. Respondent', '3. Spouse', nan, '2. Approximately equal weight'}

employment_03
{'2. Currently looking for work', '4. Retired, incapacitated, or does not work', nan, '3. Dedicated to household chores', '1. Currently Working'}

age_12
{'3. 70–79', '0. 49 or younger', '4. 80+', nan, '2. 60–69', '1. 50–59'}

urban_12
{nan, '0. <100,000', '1. 100,000+'}

married_12
{'3. Wido

In [12]:
binary_columns = []
for column in testing_df2.columns:
    if testing_df2[column].dtype == "object":
        nums = set(testing_df2[column].values)
        
        if np.nan in nums:
            nums.remove(np.nan)
        else:
            pass
        
        if len(nums) == 2:
            binary_columns.append(column)

binary_columns

['urban_03',
 'urban_12',
 'ragender',
 'sgender_03',
 'sgender_12',
 'rrelgwk_12',
 'a34_12']

In [13]:
for col in binary_columns:
    if testing_df2[col].dtype == 'object':
        # Get unique values (excluding NaN)
        vals = set(testing_df2[col].dropna().values) 
        
        # Create a mapping for these values
        mapping = {val: i for i, val in enumerate(sorted(vals))}
        
        # Add mapping for NaN
        mapping[np.nan] = np.nan
        
        # Apply the mapping to the column
        testing_df2[col] = testing_df2[col].map(mapping)

In [14]:
check = 0
for column in testing_df2.columns:
    if testing_df2[column].dtype == "object":
        print(column)
        print(set(testing_df2[column].values))
        print()
        check += 1

print(check)

age_03
{'3. 70–79', '0. 49 or younger', '4. 80+', nan, '2. 60–69', '1. 50–59'}

married_03
{'3. Widowed', '2. Separated or divorced', '1. Married or in civil union', '4. Single', nan}

edu_gru_03
{'0. No education', '2. 6 years', '3. 7–9 years', '4. 10+ years', nan, '1. 1–5 years'}

n_living_child_03
{'1. 1 or 2', '3. 5 or 6', nan, '0. No children', '2. 3 or 4', '4. 7+'}

glob_hlth_03
{'3. Good', '2. Very good', '5. Poor', nan, '4. Fair', '1. Excellent'}

bmi_03
{'4. Obese', '1. Underweight', '5. Morbidly obese', nan, '2. Normal weight', '3. Overweight'}

decis_famil_03
{'1. Respondent', '3. Spouse', nan, '2. Approximately equal weight'}

employment_03
{'2. Currently looking for work', '4. Retired, incapacitated, or does not work', nan, '3. Dedicated to household chores', '1. Currently Working'}

age_12
{'3. 70–79', '0. 49 or younger', '4. 80+', nan, '2. 60–69', '1. 50–59'}

married_12
{'3. Widowed', '2. Separated or divorced', '1. Married or in civil union', '4. Single', nan}

edu_gru

In [15]:
ordinal_cols = ["age_03", "age_12", "edu_gru_03", "edu_gru_12", "glob_hlth_03", "glob_hlth_12", "bmi_03", "bmi_12", "memory_12"]

for col in ordinal_cols:
    def process_value(val):
            if pd.isna(val):
                return np.nan
            val_str = str(val)
            if "." in val_str:
                # If there's a period, take the part before it
                return int(val_str.split(".")[0])
            else:
                # Extract digits if there is no period
                return int(''.join(filter(str.isdigit, val_str))) if val_str else np.nan
        
    # Apply the function to the entire column
    testing_df2[col] = testing_df2[col].apply(process_value)

In [16]:
check = 0
for column in testing_df2.columns:
    if testing_df2[column].dtype == "object":
        print(column)
        print(set(testing_df2[column].values))
        print()
        check += 1

print(check)

married_03
{'3. Widowed', '2. Separated or divorced', '1. Married or in civil union', '4. Single', nan}

n_living_child_03
{'1. 1 or 2', '3. 5 or 6', nan, '0. No children', '2. 3 or 4', '4. 7+'}

decis_famil_03
{'1. Respondent', '3. Spouse', nan, '2. Approximately equal weight'}

employment_03
{'2. Currently looking for work', '4. Retired, incapacitated, or does not work', nan, '3. Dedicated to household chores', '1. Currently Working'}

married_12
{'3. Widowed', '2. Separated or divorced', '1. Married or in civil union', '4. Single', nan}

n_living_child_12
{'1. 1 or 2', '3. 5 or 6', nan, '0. No children', '2. 3 or 4', '4. 7+'}

decis_famil_12
{'1. Respondent', '3. Spouse', nan, '2. Approximately equal weight'}

decis_personal_12
{'1. A lot', '3. None', nan, '2. A little'}

employment_12
{'2. Currently looking for work', '4. Retired, incapacitated, or does not work', nan, '3. Dedicated to household chores', '1. Currently Working'}

satis_ideal_12
{'2. Neither agrees nor disagrees', na

In [17]:
cat_columns = testing_df2.select_dtypes(include = 'object').columns 
testing_df2 = pd.get_dummies(testing_df2, columns=cat_columns, dummy_na=True, drop_first=False)

In [18]:
cat_columns = testing_df2.select_dtypes(include = 'bool').columns
testing_df2[cat_columns] = testing_df2[cat_columns].astype('int')

In [1]:
testing_df2

NameError: name 'testing_df2' is not defined

In [20]:
for col in testing_df2.columns:
    ratio = testing_df2[col].isna().sum() / len(testing_df2[col])
    if ratio == 0:
        pass
    else:
        print(col)
        print(ratio)
        print()

age_03
0.3162393162393162

urban_03
0.3156288156288156

n_mar_03
0.32173382173382176

edu_gru_03
0.31868131868131866

migration_03
0.3159340659340659

glob_hlth_03
0.35775335775335776

adl_dress_03
0.35744810744810745

adl_walk_03
0.31776556776556775

adl_bath_03
0.31776556776556775

adl_eat_03
0.31807081807081805

adl_bed_03
0.31776556776556775

adl_toilet_03
0.31776556776556775

n_adl_03
0.31807081807081805

iadl_money_03
0.35744810744810745

iadl_meds_03
0.35744810744810745

iadl_shop_03
0.35744810744810745

iadl_meals_03
0.35744810744810745

n_iadl_03
0.35744810744810745

depressed_03
0.35866910866910867

hard_03
0.35775335775335776

restless_03
0.35775335775335776

happy_03
0.3595848595848596

lonely_03
0.35775335775335776

enjoy_03
0.3592796092796093

sad_03
0.35744810744810745

tired_03
0.35744810744810745

energetic_03
0.3598901098901099

n_depr_03
0.358974358974359

cesd_depressed_03
0.358974358974359

hypertension_03
0.3156288156288156

diabetes_03
0.3156288156288156

resp_il

In [21]:
testing_y = []

for i, id in enumerate(ids):
    x = np.where(y_df['uid']==id)[0]
    testing_y.append(np.mean(y_df.iloc[x,2]))

y = pd.DataFrame({'composite_score':testing_y})

In [22]:
def rescale_y(data):
    return (data-0)/(384-0)


y = rescale_y(y)

In [23]:
y

,composite_score
0,0.455729
1,0.536458
2,0.397135
3,0.270833
4,0.476562
...,...
3271,0.627604
3272,0.399740
3273,0.339844
3274,0.083333


In [24]:
scaler = preprocessing.MinMaxScaler() 
scaled_X = pd.DataFrame(scaler.fit_transform(testing_df2), columns=testing_df2.columns)

In [25]:
scaled_X

,age_03,urban_03,n_mar_03,edu_gru_03,migration_03,glob_hlth_03,adl_dress_03,adl_walk_03,adl_bath_03,adl_eat_03,...,rsocact_m_12_5.4 or more times a month,rsocact_m_12_6.2 or 3 times a month,rsocact_m_12_7.Once a month,"rsocact_m_12_8.Almost Never, sporadic",rsocact_m_12_9.Never,rsocact_m_12_nan,j11_12_Concrete 2,j11_12_Mud 3,"j11_12_Wood, mosaic, or other covering 1",j11_12_nan
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.25,1.0,0.2,0.75,0.0,0.75,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3272,0.25,0.0,0.2,0.25,0.0,1.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3273,0.25,0.0,0.2,0.25,0.0,0.75,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3274,0.25,0.0,0.2,0.00,0.0,0.50,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [27]:
imputer = KNNImputer(n_neighbors=6)
X = pd.DataFrame(imputer.fit_transform(scaled_X), columns=scaled_X.columns)

In [28]:
X

,age_03,urban_03,n_mar_03,edu_gru_03,migration_03,glob_hlth_03,adl_dress_03,adl_walk_03,adl_bath_03,adl_eat_03,...,rsocact_m_12_5.4 or more times a month,rsocact_m_12_6.2 or 3 times a month,rsocact_m_12_7.Once a month,"rsocact_m_12_8.Almost Never, sporadic",rsocact_m_12_9.Never,rsocact_m_12_nan,j11_12_Concrete 2,j11_12_Mud 3,"j11_12_Wood, mosaic, or other covering 1",j11_12_nan
0,0.416667,0.333333,0.266667,0.250000,0.166667,0.708333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.333333,0.500000,0.233333,0.333333,0.000000,0.583333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.375000,0.666667,0.233333,0.458333,0.166667,0.625000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.250000,1.000000,0.200000,0.750000,0.000000,0.750000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.416667,0.833333,0.300000,0.291667,0.166667,0.625000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,0.250000,0.833333,0.233333,0.416667,0.166667,0.625000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3272,0.250000,0.000000,0.200000,0.250000,0.000000,1.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3273,0.250000,0.000000,0.200000,0.250000,0.000000,0.750000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3274,0.250000,0.000000,0.200000,0.000000,0.000000,0.500000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [29]:
from sklearn.linear_model import LinearRegression

Xtrain, Xtest, ytrain, ytest = sklearn.model_selection.train_test_split(X, y, test_size=0.2)

LRegModel = LinearRegression(fit_intercept = False)
LRegModel.fit(Xtrain, ytrain)

yhatTest = LRegModel.predict(Xtest)

print("Mean squared error (MSE) for your test subset is", mean_squared_error(ytest.values, yhatTest))
print("RMSE:", root_mean_squared_error(ytest.values, yhatTest))

Mean squared error (MSE) for your test subset is 0.011523996360477694
RMSE: 0.10734987825087504


In [30]:
def rescale(nums):
    return nums*(384-0)+0

print("RMSE:", root_mean_squared_error(rescale(ytest.values), rescale(yhatTest)))

RMSE: 41.22235324833602


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [32]:
model = Sequential([

    Input(shape=(X.shape[1],)),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(1, activation='sigmoid')

])

In [33]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae'])

In [34]:
Xtrain, Xtest, ytrain, ytest = sklearn.model_selection.train_test_split(X, y, test_size=0.2)

In [35]:
model.fit(Xtrain, ytrain, epochs=100, batch_size=32, validation_data=(Xtest, ytest))

Epoch 1/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 0.9982 - mae: 0.1897 - val_loss: 0.8428 - val_mae: 0.1336
Epoch 2/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.8048 - mae: 0.1340 - val_loss: 0.6871 - val_mae: 0.1209
Epoch 3/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.6508 - mae: 0.1177 - val_loss: 0.5505 - val_mae: 0.1168
Epoch 4/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.5190 - mae: 0.1132 - val_loss: 0.4316 - val_mae: 0.1075
Epoch 5/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.4055 - mae: 0.1058 - val_loss: 0.3350 - val_mae: 0.1073
Epoch 6/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.3124 - mae: 0.1005 - val_loss: 0.2561 - val_mae: 0.1012
Epoch 7/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2377 - mae: 0.0933 - val_loss: 0.1943 - val_mae: 0.0948
Epoch 8/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1809 - mae: 0.0922 - val_loss: 0.1484 - val_mae: 0.0967
Epoch 9/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1

In [36]:
yhatTest = model.predict(Xtest)

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [37]:
print("Mean squared error (MSE) for your test subset is", mean_squared_error(ytest.values, yhatTest))
print("RMSE:", root_mean_squared_error(ytest.values, yhatTest))

Mean squared error (MSE) for your test subset is 0.012001768737041202
RMSE: 0.10955258434670176


In [38]:
def rescale(nums):
    return nums*(np.max(testing_y)-np.min(testing_y))+np.min(testing_y)

print("RMSE:", root_mean_squared_error(rescale(ytest.values), rescale(yhatTest)))

RMSE: 35.9880238732539


In [39]:
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(Xtrain, ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [40]:
yhatTest = LRegModel.predict(Xtest)

print("Mean squared error (MSE) for your test subset is", mean_squared_error(ytest.values, yhatTest))
print("RMSE:", root_mean_squared_error(ytest.values, yhatTest))

Mean squared error (MSE) for your test subset is 0.010313024069770164
RMSE: 0.10155306036634329


In [41]:
def rescale(nums):
    return nums*(384-0)+0

print("RMSE:", root_mean_squared_error(rescale(ytest.values), rescale(yhatTest)))

RMSE: 38.99637518067583
